In [1]:
import torch
from torch.utils import data
from Dataset import *
from torch.autograd import Variable
from testPlayer import *

In [2]:
dirname = 'playlabel'

trainfilenum = 10
testfilenum = 2

files = ['data/' + dirname + '/data' + str(i) + '.pt' for i in range(trainfilenum)]
labelfile = 'data/' + dirname  + '/labels.pt'
batchsize =  128
filesize = 50000
dataset = Dataset(files, labelfile, batchsize, filesize)

testfiles = ['data/' + dirname + '/data' + str(i + trainfilenum) + '.pt' for i in range(testfilenum)]
testlabel = 'data/' + dirname + '/testlabels.pt'
testset = Dataset(testfiles, testlabel, batchsize, filesize)

In [3]:
player = PlayPlayer().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, player.parameters()), lr=0.001, betas=(0.9, 0.999))

In [4]:
# state_dict = player.state_dict()
# pretrained_dict = {k: v for k, v in pretrained.items() if k in state_dict}
# # print(pretrained.keys())
# # print(player.state_dict().keys())
# state_dict.update(pretrained_dict)
# player.load_state_dict(state_dict)

In [5]:
# print(pretrained_dict.keys())

In [6]:
max_epochs = 8

# truelabels = 0.0
# falselabels = 0.0

# dataset.reset()
# for i, data in enumerate(dataset.getbatch()):
#     inputs, labels = data
#     truelabels += sum(labels)
#     falselabels += sum(1 - np.array(labels))

In [7]:
print_each = 1000
for epoch in range(max_epochs):
    player.training = True
    print('start training epoch ' + str(epoch + 1) + '....')
    running_loss = 0.0
    dataset.reset()
    
    num = 0
    correct = 0
    
    for i, data in enumerate(dataset.getbatch()):
        
        inputs, labels = data
        inputs, labels = torch.tensor(inputs, dtype=torch.uint8).cuda(), torch.tensor(labels).cuda()
        inputs = inputs.float()
        optimizer.zero_grad()
        outputs = player(inputs)
        
        tmp = torch.argmax(outputs, dim=1) == labels
        correct += torch.sum(tmp)
        num += 128
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i + 1) % print_each == 0:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / print_each))
            print('train_acc:  %.2f' % (float(correct) * 100.0 / num))
            correct = 0
            num = 0
            running_loss = 0.0

    torch.save(player.state_dict(), 'testmodel/' + dirname[:-5] + '/model' + str(epoch + 1) + '.pt')
    
    testset.reset()
    testnum = 0
    testcorrect = 0
    with torch.no_grad():
        player.training = False
        for i, data in enumerate(testset.getbatch()):

            inputs, labels = data
            inputs, labels = torch.tensor(inputs, dtype=torch.uint8).cuda(), torch.tensor(labels).cuda()
            inputs = inputs.float()

            optimizer.zero_grad()
            outputs = player(inputs)

            tmp = torch.argmax(outputs, dim=1) == labels
            testcorrect += torch.sum(tmp)
            testnum += 128
        print('test_acc:  %.2f' % (float(testcorrect) * 100.0 / testnum))
        print()
        print()
        print()
        
print('Finished Training')

start training epoch 1....
[1,  1000] loss: 3.022
train_acc:  19.95
[1,  2000] loss: 2.620
train_acc:  27.85
[1,  3000] loss: 2.503
train_acc:  29.36
[1,  4000] loss: 2.394
train_acc:  31.13
[1,  5000] loss: 2.312
train_acc:  32.69
[1,  6000] loss: 2.257
train_acc:  33.65
[1,  7000] loss: 2.228
train_acc:  34.25
[1,  8000] loss: 2.197
train_acc:  34.60
[1,  9000] loss: 2.162
train_acc:  35.44
[1, 10000] loss: 2.132
train_acc:  35.69
[1, 11000] loss: 2.091
train_acc:  36.90
[1, 12000] loss: 2.089
train_acc:  36.64
[1, 13000] loss: 2.060
train_acc:  37.17
[1, 14000] loss: 2.034
train_acc:  37.83
[1, 15000] loss: 2.026
train_acc:  38.02
[1, 16000] loss: 2.006
train_acc:  38.55
[1, 17000] loss: 1.988
train_acc:  38.90
[1, 18000] loss: 1.980
train_acc:  39.03
[1, 19000] loss: 1.968
train_acc:  39.22
[1, 20000] loss: 1.950
train_acc:  39.52
[1, 21000] loss: 1.954
train_acc:  39.42
[1, 22000] loss: 1.933
train_acc:  40.02
[1, 23000] loss: 1.922
train_acc:  40.09
[1, 24000] loss: 1.917
train_a

FileNotFoundError: [Errno 2] No such file or directory: 'testmodel/play/model1.pt'

In [ ]:
print(truelabels)
print(falselabels)
print(TP / int(truelabels))
print((correct - TP) * 100 / int(falselabels))

In [15]:
player.load_state_dict(torch.load('model/play/bigmodel4.pt'))

In [17]:
testset = dataset
testset.reset()
testnum = 0
testcorrect = 0
with torch.no_grad():
    for i, data in enumerate(testset.getbatch()):

        inputs, labels = data
        inputs, labels = torch.tensor(inputs, dtype=torch.uint8).cuda(), torch.tensor(labels).cuda()
        inputs = inputs.float()

        testnum += 128
        
        optimizer.zero_grad()
        outputs = player(inputs)

        tmp = torch.argmax(outputs, dim=1) == labels
        testcorrect += torch.sum(tmp)
    print('test_acc:  %.2f' % (float(testcorrect) * 100.0 / int(testnum)))
    print()
    print()
    print()

test_acc:  67.29





In [23]:
dataset.reset()
cnt = 0
tot = 0
for i, data in enumerate(dataset.getbatch()):
    inputs, labels = data
    for p in inputs:
        if sum(p[-34:]) % 3 != 2:
            cnt += 1
        tot += 1
print(cnt)
print(tot)

310
4999680


In [13]:
inputs = inputs[0]

In [20]:
sum(inputs[-34:])

14

In [7]:
sum(np.zeros(3))

0.0

In [2]:
for i in range(2, 4):
    print(i)

2
3


In [8]:
a = [1, 2, 3]
print(tuple(a))

(1, 2, 3)


In [3]:
def t():
    print(a)

In [11]:
import json
dic = dict()
dic[0] = 1
print(json.dumps(dic))
print(dic)

{"0": 1}
{0: 1}


In [ ]:
{"requests":["0 0 2","1 0 2 2 2 W7 T1 T9 B2 T9 B7 F3 J1 W3 W5 B1 T4 W8 H7 H5 H8 H4 H1 H3",
             "2 T7","3 0 PLAY B7","3 1 CHI B6 W8","3 2 DRAW",
             "3 2 PLAY T3","3 3 CHI T4 B9","2 F1","3 0 PLAY T1","3 1 DRAW"],
 "responses":["PASS","PASS","PLAY B7","PASS","PASS","PASS","PASS","PASS","PLAY T1","PASS"]}

In [ ]:
{"requests":["0 1 1","1 1 1 1 1 B7 T3 B5 W4 T4 W3 T8 J2 B4 T3 W6 T1 B4 H3 H2 H6 H8","3 0 DRAW","3 0 PLAY T1","2 B8","3 1 PLAY J2","3 2 DRAW","3 2 PLAY T2","3 3 DRAW","3 3 PLAY W7","3 0 DRAW","3 0 PLAY B9","2 B6","3 1 PLAY T1","3 2 DRAW","3 2 PLAY W5","3 3 DRAW","3 3 PLAY B3","3 0 BUHUA H1","3 0 DRAW","3 0 PLAY W1","2 B9","3 1 PLAY B4","3 2 DRAW","3 2 PLAY F3","3 3 DRAW","3 3 PLAY W8","3 0 DRAW","3 0 PLAY W2","2 T2","3 1 PLAY T3","3 2 DRAW","3 2 PLAY W9","3 3 DRAW","3 3 PLAY F1","3 0 DRAW","3 0 PLAY F3","2 W1","3 1 PLAY W1","3 2 DRAW","3 2 PLAY T6","3 3 DRAW","3 3 PLAY B7","3 0 DRAW","3 0 PLAY T3","3 1 BUHUA H4","2 F4","3 1 PLAY F4","3 2 DRAW","3 2 PLAY W2","3 3 DRAW","3 3 PLAY F4","3 0 DRAW","3 0 PLAY W8","2 W9","3 1 PLAY W6","3 2 DRAW","3 2 PLAY J2","3 3 DRAW","3 3 PLAY J3","3 0 DRAW","3 0 PLAY B9","2 W7","3 1 PLAY W9","3 2 DRAW","3 2 PLAY B6","3 3 DRAW","3 3 PLAY B3","3 0 DRAW","3 0 PLAY W7","2 W5","3 1 PLAY W7","3 2 DRAW","3 2 PLAY W8","3 3 DRAW","3 3 PLAY F1","3 0 DRAW","3 0 PLAY W5","2 B8","3 1 PLAY T8","3 2 BUHUA H7","3 2 DRAW","3 2 PLAY W1","3 3 DRAW","3 3 PLAY B4","3 0 BUHUA H5","3 0 DRAW","3 0 PLAY W9","2 B2","3 1 PLAY B8","3 2 DRAW","3 2 PLAY T6","3 3 DRAW","3 3 PLAY W3","3 0 DRAW","3 0 PLAY B5","2 T8","3 1 PLAY T8","3 2 DRAW","3 2 PLAY F2","3 3 DRAW","3 3 PLAY W6","3 0 DRAW","3 0 PLAY W8","2 W6","3 1 PLAY W6","3 2 DRAW","3 2 PLAY J3","3 3 DRAW","3 3 PLAY J1","3 0 DRAW","3 0 PLAY F2","2 F1","3 1 PLAY F1","3 2 DRAW","3 2 PLAY W5","3 3 DRAW","3 3 PLAY B3","3 0 DRAW","3 0 PLAY W4","2 B2"],"responses":["PASS","PASS","PASS","PASS","PLAY J2","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY T1","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY B4","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY T3","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY W1","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY F4","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY W6","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY W9","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY W7","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY T8","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY B8","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY T8","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY W6","PASS","PASS","PASS","PASS","PASS","PASS","PASS","PLAY F1","PASS","PASS","PASS","PASS","PASS","PASS", "PASS"]}